In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import time
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
from collections import Counter
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV

In [2]:
test_df = pd.read_csv('/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/test_data.csv')
train_df = pd.read_csv('/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/train_data.csv')

In [3]:
test_df.set_index("id",inplace=True)
train_df.set_index("id",inplace=True)
test_df.fillna(-1,inplace=True)
train_df.fillna(-1,inplace=True) ###what to fill null values with
train_dummies= pd.get_dummies(train_df.select_dtypes(include=['O']))
test_dummies= pd.get_dummies(test_df.select_dtypes(include=['O']))

train = pd.concat([train_df.select_dtypes(include=['float64']),train_dummies],axis=1)
test=pd.concat([test_df.select_dtypes(include=['float64']), test_dummies],axis=1)


In [4]:
## We split the data into train and test
y = train["target"]
X =  train.drop("target", 1)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.20,
                                                    random_state=55,
                                                    stratify = y)

In [5]:
lrcv_clf = LogisticRegressionCV(scoring="roc_auc", cv=10, Cs=100, n_jobs=-1)

lrcv_clf.fit(X_train, y_train)

LogisticRegressionCV(Cs=100, class_weight=None, cv=10, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           refit=True, scoring='roc_auc', solver='lbfgs', tol=0.0001,
           verbose=0)

In [6]:
y_estimated_test = lrcv_clf.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test, y_estimated_test)

0.62605663212695006

In [7]:
### Run to Submit
submission_example = pd.DataFrame({"prediction": lrcv_clf.predict_proba(test)[:, 1]}, index=test.index) 
submission_example.to_csv("./submission_exampleLogReg.csv")

In [8]:
param_distributions = {
    "learning_rate": stats.uniform(0.0001, 0.5),
    "n_estimators": stats.randint(100, 500),
    "max_depth": stats.randint(2, 20),
    "min_samples_split": stats.uniform(0, 1),
    "min_samples_leaf": stats.uniform(0, 0.5),
    "min_weight_fraction_leaf": stats.uniform(0, 0.5),
    "subsample":stats.uniform(0.2, 0.8),
    "max_leaf_nodes": stats.randint(4, 1000)    
}

gbm_clf = GradientBoostingClassifier()

r_grid = RandomizedSearchCV(gbm_clf, 
                            param_distributions=param_distributions,
                            cv=5,
                            n_jobs=-1,
                            n_iter=100, ##iterations
                            verbose=1)

r_grid.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 28.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 32.1min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x114d40cf8>, 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x114d5fd68>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x114d5fac8>, 'min_samples_split': <scip...0x114d655c0>, 'max_leaf_nodes': <scipy.stats._distn_infrastructure.rv_frozen object at 0x114d65748>},
          pre_dispatch='2*n_j

In [9]:
y_estimated_test = r_grid.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test, y_estimated_test)

0.62713122517754283

In [10]:
### Run to Submit
submission_example = pd.DataFrame({"prediction": r_grid.predict_proba(test)[:, 1]}, index=test.index) 

submission_example.to_csv("./submission_exampleGBM.csv")

In [11]:
import numpy as np
import pandas as pd
from sklearn import metrics

YOUR_FILE = "/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/submission_exampleLogReg.csv"
TEST_FILE = "/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/test_data.csv"

submission_df = pd.read_csv(YOUR_FILE, index_col=["id"])
test_df = pd.read_csv(TEST_FILE, index_col=["id"])

fake_test_target = pd.DataFrame({"target": np.random.choice([0,1], size=test_df.shape[0])}, index=test_df.index)
submission_df["Truth"] = fake_test_target["target"]

metrics.roc_auc_score(submission_df["Truth"], submission_df["prediction"])

0.50126695531691989

In [12]:
YOUR_FILE = "/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/submission_exampleGBM.csv"
TEST_FILE = "/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/test_data.csv"

submission_df = pd.read_csv(YOUR_FILE, index_col=["id"])
test_df = pd.read_csv(TEST_FILE, index_col=["id"])

fake_test_target = pd.DataFrame({"target": np.random.choice([0,1], size=test_df.shape[0])}, index=test_df.index)
submission_df["Truth"] = fake_test_target["target"]

metrics.roc_auc_score(submission_df["Truth"], submission_df["prediction"])

0.50427370847559727